In [2]:
import joblib
import pandas as pd
from better_profanity import profanity

players = pd.read_csv("data/players.csv")

player_rounds = pd.read_csv("data/player_rounds.csv")

valid_ids = pd.read_csv("data/valid_ids.csv")

push_stats = pd.read_csv("data/push_stats.csv")

teams = pd.read_csv("data/teams.csv")

healspread = pd.read_csv("data/healspread.csv")

model_ready_data = joblib.load("data/pkls/model_ready_data_dict.pkl")
X = model_ready_data['X']

In [ ]:
X

In [11]:
"cpcpm" in X.columns

False

* Turn all offclass kda columns into offclass

In [ ]:
offclass = ["heavyweapons",'spy','sniper','pyro','engineer']
pick_class = ['sniper','spy']


In [ ]:
a = model_ready_data["X"]



Index(['scout_dapd_1', 'soldier_dapd_1', 'scout_dapm_1', 'soldier_dapm_1',
       'scout_kpd_1', 'soldier_kpd_1', 'scout_offclass_pct_1',
       'scout_kill_pct_1', 'soldier_kill_pct_1', 'scout_deaths_pct_1',
       ...
       'medic_avg_time_before_healing', 'medic_avg_time_to_build',
       'medic_avg_uber_length', 'medic_biggest_advantage_lost',
       'medic_deaths_with_95_99_uber_rate',
       'medic_deaths_within_20s_after_uber_rate', 'medic_drops_rate',
       'medic_exchanges_not_initiated_rate', 'medic_exchanges_initiated_rate',
       'medic_uberspm'],
      dtype='object', length=227)

* Fix healspread

In [ ]:
healspread = pd.read_csv("data/healspread.csv")

healspread['steamid'] = healspread['healer'].copy()

players_info = players[['steamid','id','team','name','primary_class']].copy()

players_info.columns = ['steamid', 'id','team','medic_name','primary_class']

healspread = healspread.merge(players_info,on = ['id','steamid'])

healspread.drop('primary_class',axis = 1,inplace= True)

healspread['steamid'] = healspread['healed'].copy()

players_info.drop('team',axis = 1,inplace = True)
players_info.columns = ['steamid', 'id', 'healed_name', 'primary_class']

healspread = healspread.merge(players_info,on = ['id','steamid'])

healspread.drop('steamid',axis = 1,inplace = True)

,Unnamed: 0,id,healer,healed,value,steamid,team,medic_name,healed_name,primary_class
0,0,3895159,[U:1:72768156],[U:1:109958665],10222,[U:1:109958665],Blue,Rapscallion,Mr.Causa,demoman
1,1,3895159,[U:1:72768156],[U:1:157883802],4665,[U:1:157883802],Blue,Rapscallion,gotten,soldier
2,2,3895159,[U:1:72768156],[U:1:131400746],5608,[U:1:131400746],Blue,Rapscallion,Alexandros,scout
3,3,3895159,[U:1:72768156],[U:1:228457023],5323,[U:1:228457023],Blue,Rapscallion,Pinaepple,scout
4,4,3895159,[U:1:72768156],[U:1:229402223],7692,[U:1:229402223],Blue,Rapscallion,Nicholas,soldier
...,...,...,...,...,...,...,...,...,...,...
66,5,3893364,[U:1:206809878],[U:1:79625875],5351,[U:1:79625875],Red,speedy,deadlyseed_,demoman
67,6,3893364,[U:1:206809878],[U:1:113271007],4056,[U:1:113271007],Red,speedy,Zam,soldier
68,7,3893364,[U:1:206809878],[U:1:91477879],3952,[U:1:91477879],Red,speedy,yosh,scout
69,8,3893364,[U:1:206809878],[U:1:208995341],3297,[U:1:208995341],Red,speedy,branslam,scout


### Player Overview Tables ###

* Basic Stats Long

In [57]:
normal_cols = ['kills','deaths','assists','dmg','dmg_real','dt','dt_real','hr']
pct_cols = ['kill_pct','deaths_pct','assists_pct','dmg_pct','dmg_real_pct','dt_pct','dt_real_pct','hr_pct']

both_cols = ['id','team','steamid','name','primary_class']

normal_df = players[both_cols + normal_cols].copy()
pct_df = players[both_cols + pct_cols].copy()

pct_df.columns = normal_df.columns

normal_df['coltype'] = "Raw"
pct_df['coltype'] = "Pct of Team"


long_player = pd.concat([normal_df,pct_df])

long_player = long_player.fillna(0)

long_player.to_csv("data/long_player_stats.csv")

* Class KDA Long

In [58]:
class_cols = [col for col in players.columns if "class_kda" in col]

class_kill = [col for col in class_cols if "kills" in col and "kdapd" not in col]

class_deaths= [col for col in class_cols if "deaths" in col and "kdapd" not in col]

class_assists = [col for col in class_cols if "assists" in col and "kdapd" not in col]

class_kill_pd = [col for col in class_cols if "kills" in col and "kdapd" in col]

class_deaths_pd = [col for col in class_cols if "deaths" in col and "kdapd" in col]

class_assists_pd = [col for col in class_cols if "assists" in col and "kdapd" in col]

keep_cols = ['id','team','steamid','name','primary_class']



class_dict = {
    "Kills" : class_kill,
    "Deaths" : class_deaths,
    "Assists" : class_assists,
    "K/D" : class_kill_pd,
    "A/D" : class_assists_pd,
    "Death Rate" : class_deaths_pd
}

long_class = pd.DataFrame()
for key in class_dict.keys():
    col_type = key
    l = class_dict[key]
    df = players[keep_cols + l].copy()
    df.columns = keep_cols + [col.split("_")[0] for col in df.columns if col not in keep_cols]
    df['col_type'] = col_type
    long_class = pd.concat([long_class,df])

long_class = long_class.fillna(0)

long_class.to_csv("data/class_kda_long.csv")

* Attach player names to player_rounds

In [64]:
names = players[['id','steamid','name']]

player_rounds = player_rounds.merge(names, on = ['id','steamid']).drop("Unnamed: 0",axis= 1)

player_rounds.to_csv("data/player_rounds.csv")

* Quantile Columns

In [ ]:
# List out columns
quant_cols = [
    "kills",
    "assists",
    "dmg",
   'dapm',
   'kpd',
   'offclass_pct',
   'kill_pct',
   'deaths_pct',
   'dmg_pct',
   'dmg_real_pct',
   'cpc_pct',
   'ka_pct',
   'assists_pct',
   'dt_pct',
   'dt_real_pct',
   'hroi',
   'assistspd',
   'demoman_kills_class_kdapd',
   'scout_kills_class_kdapd',
   'soldier_kills_class_kdapd',
   'medic_kills_class_kdapd',
    'demoman_deaths_class_kdapd',
   'scout_deaths_class_kdapd',
   'soldier_deaths_class_kdapd',
    'medic_deaths_class_kdapd',
   'dtpm',
   'dt_realpm',
   'healpm',
   'medkits_hppm', 
   'hrpm', 
   'deathspm'

]

# Only grab valid ids to make quanitles on
sub_players = players[players['id'].isin(valid_ids['id'])].copy()
sub_players = sub_players[['id',"primary_class",'steamid']+ quant_cols]

# loop through classes and construct the quantile sets
ranked_df = pd.DataFrame()
for class_name in sub_players['primary_class'].unique():
    sub_class = sub_players[sub_players['primary_class'] == class_name].copy()
    
    binding_df = sub_class[['id',"primary_class",'steamid']].copy()
    sub_class.drop(['id',"primary_class",'steamid'],axis = 1,inplace= True)
    
    sub_class = sub_class.rank(pct = True)
    sub_class = pd.concat([binding_df,sub_class],axis = 1)
    
    ranked_df = pd.concat([ranked_df,sub_class])
    
# Rename columns
ranked_df.columns = ['id','primary_class','steamid'] + [col + '_quantile' for 
                                                        col in ranked_df.columns if
                                                        col not in ['id','steamid','primary_class']]

ranked_df.to_csv("data/players_quantile.csv")
